# Dewrapping lines

In [80]:
#get authorization from google drive
#from google.colab import drive
#drive.mount('/content/drive')

In [81]:
!ls "input/nypl_1880_81_starred.txt"

input/nypl_1880_81_starred.txt


In [82]:
#load data
import glob # import glob to retrieve files/pathnames matching a specified pattern
file_path = "input/nypl_1880_81_starred.txt"
with open(file_path) as f:
  data = f.readlines()  

In [83]:
print(data[1:10])

['Z***\n', 'Zoller Ignatz, shoes, 106 W. 24th\n', 'Zoller John, tailor. 221 Ninth av.\n', 'Zoller Robert W. birds, 5 N. William\n', 'Zoller Stephen, molder, h r 504 W. 55th\n', 'Zollfrai Sarah, nurse, h 934 First av.\n', 'Zollfroi Abraham, pedlar, h 202 Seventh\n', 'Zollfrey Moses, cutter, h 208 E. 49th\n', 'Zollie Louis, h r 10 Dominick\n']


In [84]:
import pandas as pd
pd.DataFrame(data).shape

(307677, 1)

There are 307,677 rows

## Divide records into different sections based on first letter

In [85]:
d = {}
value = []
i = 0
for line in data:
  line = line.strip()
  if line:
    if len(line) < 6: 
      if line[-3:] == "***": 
        # save previous value
        if value:
          if value[0] == "'":
            value = value[1:]
          d[key] = value
        
        key = line
        value = []
    else:
      value.append(line)

In [86]:
d.items

<function dict.items>

## Combline lines that belong to the same record

Criteria:

1) First letter of next line not equal to the initial letter in this section

2) Special cases in the start of next line: B’klyn, B'way, E., W.

In [87]:
for key,value in d.items():
  initial = key[0]
  new_value = []
  n = len(value)
  i = 0
  
  while i < n:
    if i < n-1:
      if (value[i+1][0] != initial and value[i+1][0] != "*") or (value[i+1][0:2] in ["B’", "E.", "W."]) or (value[i+1][0:2] == ". "):
        if value[i][-1] == "-":
          s = value[i][:-1] + value[i+1]
        else:
          s = value[i] + ' ' + value[i+1]
        new_value.append(s)
        i += 2
      else:
        new_value.append(value[i])
        i += 1 
    else:
      if value[i][0] == initial:
        new_value.append(value[i])
      i += 1
  
  d[key] = new_value      

In [88]:
d

{'Z***': ['Zoller Ignatz, shoes, 106 W. 24th',
  'Zoller John, tailor. 221 Ninth av.',
  'Zoller Robert W. birds, 5 N. William',
  'Zoller Stephen, molder, h r 504 W. 55th',
  'Zollfrai Sarah, nurse, h 934 First av.',
  'Zollfroi Abraham, pedlar, h 202 Seventh',
  'Zollfrey Moses, cutter, h 208 E. 49th',
  'Zollie Louis, h r 10 Dominick',
  'Zollikoffer Oscar, pres. 1545) B’way, h 210 W. 46th',
  'Zollikoffer Oscar F. sec. 1549 B’way, h 65 W. 54th',
  'Zollinger Caroline, wid. William, h 241 W. 53d',
  'Zollmann Leopold, shoes, 344 E. Houston',
  'Zollner Charles, express, h 315 W. 35th',
  'Zollver Elizabeth, dressmkr. 142 Third',
  'Zollver John, tailor, h 142 Third',
  'Zolnowski Vincent, physician, h 33 W. 16th',
  'Zolvert Auguste, tailor, h 41 Marion',
  'Zoncada Frederic W. tinner, h 309 Pearl',
  'Zong Joseph, driver, h 203 W. 30th',
  'Zonino Michael, laborer, h 203 S. 5th av.',
  'Zons Christian T. hats, 121 Third av.',
  'Zonzy John, candy, 87 Oliver',
  'Zook Stephen, labor

In [89]:
# remove "(see adv...."
for key,value in d.items():
  n = len(value)
  for i in range(n):
    ls = value[i].split()
    n2 = len(ls)
    for j in range(n2):
      if "see" in ls[j]:
        value[i] = ' '.join(ls[:j])
        break

## Identify business vs person records

Made an update on 27/03/20 to change the index notation.

* The first letter of Company names were being removed by the function
    * ZUCCA BROTHERS
    * ARNAULT PETER
    * APPELL JACOB
    * APPLETON’S AMERICAN CYCLOPG5DIA
    * BURTON A. B 

In [90]:
# add an index to each record
# in another dictionary, for each index specify whether it's business or person record
index = 1
d_bp = []
for key,value in d.items():
  n = len(value)
  for i in range(n):
    ls = value[i].split()
    if ls:
      if ls[0][0] == "*" or (len(ls[0]) > 1 and ls[0].isupper()): # Upper case as well
        value[i] = value[i][0:] # Changed from [0:] to [1:]
        d_bp.append([str(index), "b"])
      else:
        d_bp.append([str(index), "p"])
    
      value[i] = str(index) + "\t" + value[i]
      index += 1

## Save files

In [91]:
file = open("input/nypl_1880_81_starred_clean.txt", "w")
for key,value in d.items():
  file.write(key + '\n')
  for record in value:
    file.write(record + '\n')
  file.write('\n')

file.close()

file2 = open("input/index_business_person.txt", "w")
for item in d_bp:
  file2.write(item[0] + ' ' + item[1] + '\n')
  
file2.close()